In [1]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
# import project_heart as ph
from project_heart.modules.geometry import Geometry
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

In [2]:
from project_heart.enums import *

In [3]:
filepath = Path("../sample_files/lvhexmesh.vtk")
lv = LV.from_pyvista_read(filepath)
lv.plot()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [4]:
lv.identify_surfaces(
  endo_epi_args=dict(threshold=90.0),
  apex_base_args=dict(ab_ql=0.04, ab_qh=0.69),
  aortic_mitral_args=dict(a1=0.4,
                          a2 = 0.5,
                          a3=0.3,
                          a4=70,
                          a5=132,
                          m1=0.17,
                          m2=0.02,
                          m3=0.07,
                          m4=0.333
                          )
)

In [5]:
lv.plot("mesh", scalars=LV_MESH_DATA.SURFS.value, 
        vnodes=[
          (LV_VIRTUAL_NODES.MITRAL, "green"),
          (LV_VIRTUAL_NODES.AORTIC, "red"),
          (LV_VIRTUAL_NODES.AORTIC_BORDER, "purple"),
          ],
        cmap="tab20")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [25]:
aortic_bc = lv.create_spring_rim_bc(LV_SURFS.BORDER_AORTIC.value,
                                    dist_from_c=5.0,
                                    r_alpha=0.7)
mitral_bc = lv.create_spring_rim_bc(LV_SURFS.BORDER_MITRAL.value,
                                    dist_from_c=5.0,
                                    r_alpha=1.0)

aortic_epi_bc = lv.create_spring_rim_bc(LV_SURFS.EPI_AORTIC.value,
                                    dist_from_c=0.0,
                                    r_alpha=1.5)

In [26]:
atr_rim_string_lines = lv.get_springs_pts_for_plot(
  lv.points(mask=lv.get_nodeset(aortic_bc[LV_RIM.REF_NODESET.value])),
  aortic_bc[LV_RIM.NODES.value],
  aortic_bc[LV_RIM.RELATIONS.value], 
  1
  )
mtr_rim_string_lines = lv.get_springs_pts_for_plot(
  lv.points(mask=lv.get_nodeset(mitral_bc[LV_RIM.REF_NODESET.value])),
  mitral_bc[LV_RIM.NODES.value],
  mitral_bc[LV_RIM.RELATIONS.value], 
  1
  )

atr_rim_string_lines = lv.get_springs_pts_for_plot(
  lv.points(mask=lv.get_nodeset(aortic_bc[LV_RIM.REF_NODESET.value])),
  aortic_bc[LV_RIM.NODES.value],
  aortic_bc[LV_RIM.RELATIONS.value], 
  1
  )

In [27]:
plotter = pv.Plotter(lighting='three lights')
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
plotter.add_points(lv.get_virtual_node("MITRAL"), color="green", point_size=300)
plotter.add_points(lv.get_virtual_node("AORTIC"), color="red", point_size=300)
plotter.add_points(aortic_bc[LV_RIM.NODES.value], color="red", point_size=300)
plotter.add_points(aortic_bc[LV_RIM.CENTER.value], color="red", point_size=300)

plotter.add_points(mitral_bc[LV_RIM.NODES.value], color="green", point_size=300)
plotter.add_points(mitral_bc[LV_RIM.CENTER.value], color="green", point_size=300)

plotter.add_mesh(atr_rim_string_lines, color="red", opacity=0.5, show_edges=False)
plotter.add_mesh(mtr_rim_string_lines, color="green", opacity=0.5, show_edges=False)

plotter.add_points(aortic_epi_bc[LV_RIM.NODES.value], color="pink", point_size=300)
plotter.add_mesh(atr_epi_rim_string_lines, color="pink", opacity=0.8, show_edges=False)


plotter.add_mesh(lv.mesh, 
                  scalars="LV_SURFS", 
                  cmap="Set2", 
                  opacity=1.0, 
                  show_edges=False,
                  ambient=0.2, 
                  diffuse=0.5, 
                  specular=0.5, 
                  specular_power=90,
                  #  smooth_shading=True,
                  )
plotter.enable_shadows()
plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…